In [1]:
%pylab inline

import pandas as pd
import seaborn as sns
sns.set()
import pickle
from sklearn.neighbors import KNeighborsClassifier , RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split , cross_val_score, StratifiedShuffleSplit, cross_val_predict
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, LabelEncoder
from sklearn import metrics
from sklearn.dummy import DummyClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import (RandomForestClassifier, VotingClassifier, BaggingRegressor)
import xgboost as xgb

Populating the interactive namespace from numpy and matplotlib


In [2]:
with open('atxac_de.pickle', 'rb') as read_aacde:
    atxac_df = pickle.load(read_aacde)

In [4]:
for idx,value in enumerate(atxac_df['Adoption']):
    if value == 'Adoption':
        atxac_df.at[idx,'Adoption_Binary']=1
    else:
        atxac_df.at[idx,'Adoption_Binary']=0

In [20]:
stdsc = StandardScaler()
X_train_val['TimeSinceIntake'] = stdsc.fit_transform(X_train_val[['TimeSinceIntake']])

<ipython-input-20-c2311b1504fb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_val['TimeSinceIntake'] = stdsc.fit_transform(X_train_val[['TimeSinceIntake']])


In [19]:
stdsc = StandardScaler()
X_train_val['AgeYrs'] = stdsc.fit_transform(X_train_val[['AgeYrs']])

<ipython-input-19-09a737109838>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_val['AgeYrs'] = stdsc.fit_transform(X_train_val[['AgeYrs']])


In [5]:
X = atxac_df.drop(columns=['Outcome_Type','Updated_Outcome_Type','Adoption','Adoption_Binary'])
y = atxac_df['Adoption_Binary']

In [15]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=.2, random_state=88,stratify=y)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=24,stratify=y_train_val)

In [11]:
#Stand. - 'AgeYrs' , 'TimeSinceIntake'
num_features = ['AgeYrs' , 'TimeSinceIntake']
#OHE - 'Sex','Spayed_Neutered','Sex_Upon_Outcome', 'OutcomeBreed' , 'OutcomeColor','Intake_Type','Intake_Condition','IntakeMonth','No_Known_Owner'
cate_features = ['Sex','Spayed_Neutered','Sex_Upon_Outcome', 'OutcomeBreed' , 'OutcomeColor','Intake_Type','Intake_Condition','IntakeMonth','No_Known_Owner']
combined = num_features+cate_features

In [21]:
coltransf = make_column_transformer((OneHotEncoder(handle_unknown='ignore'),cate_features),remainder='passthrough')

In [22]:
X_breakout = coltransf.fit_transform(X_train_val)

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

In [7]:
gbm = xgb.XGBClassifier( 
        n_estimators=30000,
        max_depth=4,
        objective='binary:logistic', #new objective
        use_label_encoder=False,
        learning_rate=.05, 
        subsample=.8,
        min_child_weight=3,
        colsample_bytree=.8)

In [23]:
eval_set=[(X_train,y_train),(X_val,y_val)]

fit_model = gbm.fit(
                X_train, y_train, 
                eval_set=eval_set,
                eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                early_stopping_rounds=50,
                verbose=False)

accuracy_score(y_test, gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit))

ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.Animal_ID, Name, IntakeDateTime, IntakeMonthYear, Found_Location, Intake_Type, Intake_Condition, Animal_Type, Sex_Upon_Intake, Age_Upon_Intake, Breed, Color, Outcome_Animal_ID, Outcome_Name, OutcomeDateTime, OutcomeMonthYear, DOB, Outcome_Subtype, Outcome_Animal_Type, Sex_Upon_Outcome, Outcome_Age, OutcomeBreed, OutcomeColor, TimeSinceIntake, Sex, Spayed_Neutered